In [1]:
import gsw
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import xarray as xr

from acspype import ACSDev
from acspype import ACSTSCor
import acspype.processing as acsproc
import acspype.qaqc as acsqaqc
from acspype.tscor import ACSTSCor

In [2]:
data_filepath = '../dev_tools/test_files/TEST_SHIMADA_202405.nc'
acs = xr.open_dataset(data_filepath)

In [3]:
dev_filepath = '../dev_tools/test_files/ACS-00412_2023-05-10.dev'
dev = ACSDev(dev_filepath)

In [4]:
dev.serial_number

'ACS-00412'

In [6]:
acs.attrs['acs_serial_number']

'ACS412'